# **2.B FEATURE EXTRACTION**
Phishing URLs only

#### The objective of this notebook is to collect data and save it as a CSV file for Feature Extraction.

* Lexical Features
* Whois Features
* Popularity Features

#### This project is worked on Jupyter Notebook 

In [1]:
import pandas as pd
from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
import whois
import urllib.request
import time
import socket
from urllib.error import HTTPError
from datetime import datetime

In [2]:
#import dataset

legitimate_urls = pd.read_csv("/Users/jillkathleen/Desktop/Phishing-Analysis-Detection/Back-End/Feature-Extraction-ntbk/FE-more-data/Webpages_Classification_test_data.csv")



In [3]:
legitimate_urls

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,0,http://www.dutchthewiz.com/freeware/,36,175.67.214.68,China,com,complete,yes,38.5,0.000,"Decay suggest in 1315.. Current constitution, ...",good
1,1,http://www.collectiblejewels.com,32,188.120.171.121,Sweden,com,incomplete,yes,187.0,0.000,breast addict nudger whash ky darkie catholics...,good
2,2,http://www.deadlinedata.com,27,193.51.170.1,France,com,complete,yes,31.0,0.000,Nato's military stoic philosophy says to accep...,good
3,3,http://www.mil.fi/maavoimat/kalustoesittely/00...,56,13.237.35.44,Australia,fi,complete,yes,152.0,0.000,Night being newton. according to the formation...,good
4,4,http://www.avclub.com/content/node/24539,40,220.193.62.89,China,com,complete,yes,150.0,0.000,34 per two children. if we exercise simple pra...,good
...,...,...,...,...,...,...,...,...,...,...,...,...
361929,361929,http://www.allnetporn.com/asians/t17_43/netver...,55,200.200.169.58,Brazil,com,incomplete,no,407.7,289.467,roundeye poverty welfare lovejuice shitter pec...,bad
361930,361930,http://members.shaw.ca/brianholden,34,217.114.245.202,Russia,ca,complete,yes,163.5,0.000,"Be mixed southeast alaska.. Corals, and 1936 s...",good
361931,361931,http://members.tripod.com/sjfloats/,35,191.49.186.181,Brazil,com,incomplete,yes,32.0,0.000,"For 32 guadix, spain, as well as china and ind...",good
361932,361932,http://www.holyokemachine.com/,30,202.100.138.248,China,com,complete,yes,0.0,0.000,Common arrangement involves determining what p...,good


In [4]:

legitimate = pd.read_csv("/Users/jillkathleen/Desktop/Phishing-Analysis-Detection/Back-End/Feature-Extraction-ntbk/FE-more-data/Webpages_Classification_test_data.csv",  usecols = ["url"] )


In [5]:
legitimate

,url
0,http://www.dutchthewiz.com/freeware/
1,http://www.collectiblejewels.com
2,http://www.deadlinedata.com
3,http://www.mil.fi/maavoimat/kalustoesittely/00...
4,http://www.avclub.com/content/node/24539
...,...
361929,http://www.allnetporn.com/asians/t17_43/netver...
361930,http://members.shaw.ca/brianholden
361931,http://members.tripod.com/sjfloats/
361932,http://www.holyokemachine.com/


In [4]:

#legit = legitimate.sample(n = 6000, random_state = 12).copy()

#legitimate_urls = legit.reset_index(drop=True)
#legitimate_urls


#legitimate_urls.columns = ['url']
#legitimate_urls.head()

## 2.1 Lexical Features

* URL Length 
* URL Shortening Services “TinyURL”
* URL Presence of "@" Symbol
* URL Presence of special characters : _ ? = & etc
* URL Suspicious words (security sensitive words)
* URL Digit Count
* URL Protocol Count (http / https)
* URL Dot Count
* URL Hyphen Count
* Domain presence of IP Address
* Domain presence of hyphen / prefix or Suffix
* Sub Domain and Multi Sub Domains Count
* Redirecting "//" in URL (// position)
* URL presence of EXE


In [5]:
#class FeatureExtraction:
#    def __init__(url):
#        pass

# 1.Extracts domain from the given URL
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain
    
# 2.Checks for IP address in URL (Have_IP)
def ip_address(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip
    
# 3.Checks the presence of @ in URL (Have_At)
def have_at_symbol(url):
    if "@" in url:
        at = 1 
    else:
        at = 0   
    return at
    
# 4.Finding the length of URL and categorizing (URL_Length)
def long_url(url):
    if len(url) < 54:
        length = 0    
    else:
        length = 1    
    return length

# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth
        
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
    
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https://|http://' in domain:
        return 1
    else:
        return 0

    
# 8. Checking for Shortening Services in URL (Tiny_URL) 
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                    'tr\.im|link\.zip\.net', url)
    if match:
        return 1               # phishing
    else:
        return 0               # legitimate
    
    
    
    
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)     
def prefix_suffix_separation(url):
    if "-" in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
    
# 10. DNS Record 

    
# 11.Web traffic (Web_Traffic)
def web_traffic(url):
    try:
        url = urllib.parse.quote(url)
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
        rank = int(rank)
    except TypeError:
        return 1
    if rank <100000:
        return 1
    else:
        return 0
        
# 12.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
        try:
            creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            age = 1
        else:
            age = 0
    return age

# 13.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
    expiration_date = domain_name.expiration_date
    if isinstance(expiration_date,str):
        try:
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if (expiration_date is None):
        return 1
    elif (type(expiration_date) is list):
        return 1
    else:
        today = datetime.now()
        end = abs((expiration_date - today).days)
    if ((end/30) < 6):
        end = 0
    else:
        end = 1
    return end

# 14. Dot count
def dot_count(url):
    if url.count(".") < 3:
        return 0            # legitimate
    elif url.count(".") == 3:
        return 1            # suspicious
    else:
        return 1            # phishing
        
    
# 14. Special characters count
def specialcharCount(url):
    cnt = 0
    special_characters = [';','+=','_','?','=','&','[',']','/',':']
    for each_letter in url:
        if each_letter in special_characters:
            cnt = cnt + 1
    return cnt


# 15. 
def subdomCount(url):

    # separate protocol and domain then count the number of dots in domain
    
    domain = url.split("//")[-1].split("/")[0].split("www.")[-1]
    if(domain.count('.')<=1):
        return 0
    else:
        return 1

In [6]:
#Function to extract features
def featureExtraction(url,label):
    
    features = []
    
    features.append(getDomain(url))
    features.append(ip_address(url))
    features.append(have_at_symbol(url))
    features.append(long_url(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(shortening_service(url))
    features.append(prefix_suffix_separation(url))
  
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1
        
    features.append(dns)
    features.append(web_traffic(url))
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))
    
    features.append(dot_count(url))
    features.append(specialcharCount(url))
    features.append(subdomCount(url))
    

    features.append(label)
    
    
    return features

In [7]:
feature_names = ['domain', 'ip_present', 'at_present', 'url_length', 'url_depth','redirection', 
                      'https_domain', 'short_url', 'prefix/suffix', 'dns_record', 'web_traffic', 
                      'domain_age', 'domain_end', 'dot_count', 'specialchar_count','subdom_count', 'label']

label = 0

In [8]:
# Extracting the features & storing them in a list
# Lexical Features

# starting time
start_time = time.time()
print('\n')
print('Begin feature extraction for benign dataset.... \n')

##===================================##


#Extracting the feautres & storing them in a list
legit_features = []
rows = len(legitimate_urls['url'])
label = 0

for i in range(0, rows):
    url = legitimate_urls['url'][i]
    print(i), print(url)
    
    
    legit_features.append(featureExtraction(url,label))

    
##===================================##

elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
print('\n')
print(f"Runtime: Feature Extraction for legitimate dataset took:  {elapsed}")


print('\n\n\n\n')
print("***Legitimate Features")




Begin feature extraction for benign dataset.... 

0
wild.nhl.com/club/page.htm?id=64529
1
torontohockey.net/hockey-schools/alain-lemieux-s-mini-hockey-development-camp-london.html
2
yasas.com/videos/greek-videos-details.asp?id=5
3
canadawideflowers.ca/
4
wn.com/Eugene_Oregon_Mixed_Martial_Arts
5
section15.ca/features/people/2001/05/09/cairine_wilson/
6
teamico-aboywithhorns.blogspot.com/
7
www.centexbel.be/Eng/lexicon_frame.htm
8
store.meniketti.com/
9
www.informatik.uni-trier.de/~ley/db/conf/splc/
10
redtube.com/8080
11
daryltdixonlaw.com/feed.cfm
12
cairp.ca/about/board-of-directors/index.php
13
www.adobe.com/aboutadobe/careeropp/college/internships.html
14
youtube.com/watch?v=kx4MvFbJdXA
15
bmxsociety.com/topic/50822-patterson-factory-bike-pictures/page__st__40
16
lyricsmode.com/lyrics/0-9/3x_krazy/
17
imdb.com/name/nm0523415/
18
wral.com/specialreports/michelleyoung/story/9920227/
19
myballard.com/forum/topic.php?id=3273
20
hiphopgossips.com/ellen-page-and-drew-barrymore-kiss-on-

165
123people.com/s/guy+lapointe
166
www.dragonslairii.com/Arcade/MAMEv1.asp
167
commons.wikimedia.org/wiki/%C3%89cole_Sainte-Marguerite-Bourgeoys
168
raidersandrebels.com/2010/04/papineau-and-nelson-lower-canada.html
169
theatrehistory.com/
170
cityroom.blogs.nytimes.com/2007/08/17/revisiting-joe-goulds-secret/
171
maxalbums.com/index.php?artist=Eminem+Hailies+Revenge&album=
172
www.repmesa.com/linecard.html
173
encyclopedia.com/topic/James_Wilson_Morrice.aspx
174
btjunkie.org/search?q=My+Beautiful+Dark+Twisted+Fantasy
175
sports.yahoo.com/soccer/news?slug=reu-worldzealandfailures
176
hypebeast.com/2009/04/tomoya-nagase-for-neighborhood-2009-springsummer/
177
kckchamber.com/kckc_info_staff_members.cfm
178
manta.com/c/mtkh100/pete-ward-travel-cruise-center
179
www.knosof.co.uk/ctoa.html
180
history.navy.mil/photos/sh-usn/usnsh-p/aph2.htm
181
tradebit.com/filedetail.php/38250025-jay-mcshann-early-bird-jay
182
mylife.com/harveyb76
183
www.rebol.org/cgi-bin/cgiwrap/rebol/ml-index.r
184
ex

332
www.wkbradford.com/classmaster.html
333
ibikedb.net/bikes/25634-argon-18-gallium
334
answers.yahoo.com/question/index?qid=20060825211359AAIp8Zb
335
factbites.com/topics/List-of-E-Postal-Codes-of-Canada
336
linkedin.com/pub/dir/jason/cropper
337
robkurtzman.com/
338
www.angelfire.com/fl/chessninja/
339
sh.wikipedia.org/wiki/Edmund_Gwenn
340
mcshane-construction.com/board.aspx
341
onlineradio2.com/listen/Hits_FM_947
342
freepages.genealogy.rootsweb.ancestry.com/~maxineweddell/68.htm
343
psychodd1.blogspot.com/2011/10/why-i-detest-socialists.html
344
ectoguide.org/artists/mcgarrigle.kate.anna
345
en.wikipedia.org/wiki/Normand_Dupont
346
ffwdweekly.com/article/music/music-previews/big-bounce-hits-calgary/
347
superpages.com/bp/Kansas-City-MO/Kpxe-TV50-L0503298882.htm
348
ussoccer.com/News/Mens-National-Team/2009/10/Honduras-Game-Notes.aspx
349
en.wikipedia.org/wiki/Andreas_Olsson
350
latinoreview.com/news/steve-carell-to-star-in-seeking-a-friend-for-the-end-of-the-world-13093
351
nosor

494
youtube.com/watch?v=RIkoHNwwdZg
495
int.txtr.com/beagle/
496
www.deluxe.com/dlxfi/deluxe-financial-institutions.jsp
497
pktube.onepakistan.com/Gilles+Latulippe/
498
mtv.ca/movieawards/2009/
499
youtube.com/watch?v=4YHMBURw4_M
500
www.cs.cmu.edu/afs/cs.cmu.edu/user/mleone/web/language-research.html
501
classic-scifi-films.com/
502
dianamartinez.net/
503
imgacademies.com/basketball-academy/camps/summer-tournament-camp/
504
www.modulaware.com/mwbiblio.htm
505
berecruited.com/high-schools/ohio/lima/lima-central-catholic-high-school
506
content.usatoday.com/community/tags/topic.aspx?req=tag&tag=Walt%20Disney%20Pictures
507
staplesautoengineering.com/
508
edition-peters.com/composer/Aldridge-RobertLivingston
509
youtube.com/watch?v=wn0xeqjwRC0
510
facebook.com/event.php?eid=157341050998528
511
hockeyforum.com/montreal-canadiens/28102-montreal-canadiens-goal-song.html
512
lawyers.com/New-York/New-York/Andrew-Keats-29151866-a.html
513
vintagelovelies.com/main
514
youtube.com/watch?v=aguG8S

650
histopolis.com/Place/US/NY/St_Lawrence_County/Massena_Center
651
linkedin.com/pub/shawn-cormier/0/931/194
652
scaryforkids.com/scary-maze/
653
jameshough.com/
654
answers.com/library/Dictionary+of+Dance-letter-1C-first-51
655
bellemerefarm.com/pages/bully_bullion
656
sports.yahoo.com/nfl/news?slug=ycn-10330751
657
en.wikipedia.org/wiki/Britten-Norman_Trislander
658
amazon.com/Ladies-Gentlemen-Are-Floating-Space/product-reviews/B000002VTE
659
en.wikipedia.org/wiki/Adam_(name)
660
www.xmission.com/~daina/pinball.html
661
ratings.fide.com/card.phtml?event=5200750
662
amazon.com/L-Anthologie-1943-1966-Alys-Robi/dp/B0002CHLLC
663
www.ciras.iastate.edu/toc/
664
en.wikipedia.org/wiki/Paul_Adey
665
hotels-world.com/tp.hotel/hotel.7135/Alsace.Mulhouse/Best_Western_Intl/Best_Western_Hotel_De_La_Bourse.htm
666
amazon.com/Warning-Green-Day/dp/B00004XQP4
667
baseball-reference.com/players/t/tuckery01.shtml
668
vitals.com/doctors/Dr_Craig_Huber.html
669
1383870.golden-harvest.stone.cc/
670
hot-p

...

8289
www.ibm.com/developerworks/xml/library/x-dita4/
8290
wwe.mytopix.com/owen_hart/
8291
championship-t-shirts.blogspot.com/2011/06/bruins-stanley-cup-championship-hats.html
8292
torrentdeluxe.com/blitz-jason-statham-french-torrent.html
8293
www.varian.net/dreamview/dreameffects/index.html
8294
www.ibiblio.org/pub/Linux/devel/lang/ada/!INDEX.html
8295
amtrak.com/
8296
larryvoyer.com/genealogy/getperson.php?personID=I72824&tree=v7_28
8297
moritzlaw.osu.edu/careerservices/guides/careers/clerkships/judges_hires.php
8298
hotflick.net/celebs/cary_lawrence.html
8299
marcopapenberg.girlshopes.com/
8300
womenshistory.about.com/library/bio/blleonowens.htm
8301
en.wikipedia.org/wiki/Children%27s_Hospital_Oakland_Research_Institute
8302
epistolae.ccnmtl.columbia.edu/woman/78.html
8303
tmi.me/hJKsv
8304
plato.stanford.edu/entries/turing-machine/
8305
retrorenovation.com/2009/07/31/1950-time-capsule-home-in-burbank-calif/
8306
mp3raid.com/music/common_go.html
8307
facebook.com/pages/Redwood-Height

8450
crazyabouttv.com/warandremembrance.html
8451
associatedcontent.com/article/5874652/the_top_20_places_to_visit_in_california.html
8452
ncbi.nlm.nih.gov/pmc/articles/PMC2375799/
8453
jobs.disneyabcjobs.com/
8454
hypem.com/item/1ftj2/Tim+Hecker+-+Sketch+9
8455
americanjobs.com/
8456
bhgmoves.wordpress.com/2011/02/28/monmouth-park-racetrack-oceanport-nj/
8457
myspace.com/allen_fieldhouse
8458
letrs.indiana.edu/cgi/t/text/text-idx?c=wright2;cc=wright2;view=text;rgn=main;idno=Wright2-0068%3C
8459
thepiratebay.org/torrent/3956615/Dj_Vadim_-_The_Soundcatcher
8460
movies.tvguide.com/more-dead-than-alive/106696
8461
onedayoneinternship.com/internships/peggy-notebaert-nature-museum/
8462
frankfurtgermany.mobi/
8463
edc.eng.cam.ac.uk/kim/people/mike-lewis.html
8464
acronyms.thefreedictionary.com/Communist+Party+of+India%2c+Marxists
8465
en.wikipedia.org/wiki/Grandview_Triangle
8466
tradingcarddb.com/ViewAllSet.cfm/sid/8754
8467
1m1f.com/Reticuli/
8468
projects.haskell.org/haskellmode-emacs/
8

8612
online-movie-viewer.com/movie/tarzan_ii_2005
8613
killerfilm.com/spotlight/read/session-9-ten-years-later-89088
8614
railforums.co.uk/showthread.php?t=52578&page=7
8615
sherdog.com/fighter/Tony-Johnson-Jr-41069
8616
www.cs.cmu.edu/~awb/pc110.html
8617
tools.ietf.org/html/rfc2571
8618
directwheelsandtires.com/Maya-MRS-141.html
8619
simple.wikipedia.org/wiki/Mark_Harmon
8620
costumeholic.blogspot.com/2011/02/otakuthon-2011-costume-project-princess.html
8621
miyako-albany.com/
8622
newsroom.lds.org/leader-biographies/elder-david-a-bednar
8623
moneyinthebananastand.com/?p=363
8624
oursportscentral.com/services/releases/?id=3965959
8625
guardian.co.uk/football/blog/2011/jul/22/copa-america-uruguay-paraguay
8626
mitglied.multimania.de/leari/home.htm
8627
aroza48.blogspot.com/
8628
publicbroadcasting.net/keywords/M.html?keepThis=true&TB_iframe=true
8629
music-db.org/rock/0910ad12.html
8630
richardandersonltd.com/
8631
wikinvest.com/stock/Sun_Life_Financial_(SLF)
8632
facebook.com/pages/G

8769
hotfrog.com/Companies/Vitelco-Employees-Federal-Credit-Union
8770
metrolyrics.com/unto-the-locust-album-machine-head.html
8771
lonelyplanet.com/japan/tokyo
8772
ebay.com/itm/Paysan-Pure-Cloth-Menstrual-Pads-NEW-/290470602459
8773
correspondencechess.com/ccca/index.htm
8774
qingdaochinaguide.com/listings/international-schools/kisq.html
8775
facebook.com/jmichael.smith51
8776
northernstars.ca/actorsabc/cherry_jonathan.html
8777
frankrodriguez.listingbook.com/
8778
docpit.com/publication/?publisher=le_journal_de_levis&pub=818b777d9e&page=4
8779
encyclopedia.com/topic/Jerzy_Neyman.aspx
8780
ironsidesworldheadquarters.blogspot.com/2009/02/re-mp-francine-lalonde-gives-notice-to.html
8781
eliteprospects.com/player.php?player=16658
8782
santaclarasingles.eventsandadventures.com/
8783
igougo.com/attractions-reviews-b18356-Cairo-Citadel_Complex.html
8784
drawnandquarterly.blogspot.com/2010_08_01_archive.html
8785
nndb.com/people/038/000085780/
8786
answers.com/topic/jean-claude-pascal
8787


8924
en-pi.facebook.com/people/John-Comeau/100000104685316
8925
bbb.org/ottawa/business-reviews/accountants/raymond-chabot-grant-thornton-llp-chartered-accountants-in-ottawa-on-3250
8926
museumca.org/creeks/MapOak.html
8927
torrentdownloads.net/torrent/1653294361/le+devoir+2011-11-07+pdf
8928
chrisrwright.co.uk/
8929
jsterrett.com/Issues/10.html
8930
www.lightflowtech.com/products.html
8931
deezer.com/fr/music/larry-carlton
8932
amazon.com/Sex-Packets-Digital-Underground/dp/B000000HHM
8933
letrs.indiana.edu/cgi/t/text/text-idx?c=wright2;cc=wright2;view=text;rgn=main;idno=wright2-0151
8934
ebay.com/itm/1933-OPC-O-Pee-Chee-NELS-STEWART-6-PSA-7-NM-/310330844638
8935
mylife.com/georgestevendavis
8936
logiquali.com/xkvmfr/czwh.php?qdgj=picture-of-edmund-emil-kemper-iii
8937
ninersnation.com/2011/11/4/2536726/49ers-at-redskins-preview-another-ugly-win
8938
fanbase.com/Iowa-State-Cyclones-Mens-Basketball
8939
examiner.com/sports-in-pittsburgh/cleveland-writer-trashes-steelers-and-pittsburgh-s

In [9]:
#Converting the list to dataframe

legitimate = pd.DataFrame(legit_features, columns= feature_names)
legitimate.head()

,domain,ip_present,at_present,url_length,url_depth,redirection,https_domain,short_url,prefix/suffix,dns_record,web_traffic,domain_age,domain_end,dot_count,specialchar_count,subdom_count,label
0,,0,0,0,3,0,0,0,0,0,1,1,1,1,4,1,0
1,,0,0,1,3,0,0,0,0,0,1,1,1,0,2,0,0
2,,0,0,0,3,0,0,0,0,0,0,1,1,0,4,0,0
3,,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0
4,,0,0,0,2,0,0,0,0,0,1,1,1,0,5,0,0


In [10]:
# Storing the extracted legitimate URLs fatures to csv file

legitimate.to_csv('/Users/jillkathleen/Desktop/Phishing-Analysis-Detection/Back-End/Extracted CSVs/from FeatureExtraction/Data-1/D2-benign-extracted.csv', index= False)

